In [52]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D


In [2]:
data = pd.read_csv("./data/SydneyHousePrices.csv")
data.head()

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house


## Part 1: Data Cleaning and Analysis

In [3]:
data.shape

(199504, 9)

In [4]:
data['Date'] = pd.to_datetime(data['Date'])
data.head(10)

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house
5,2019-05-10,6,Bilgola Plateau,2107,1610000,4.0,2,2.0,house
6,2019-04-13,7,Avalon Beach,2107,1510000,3.0,1,2.0,house
7,2019-04-11,8,Avalon Beach,2107,1500000,5.0,4,4.0,house
8,2019-04-11,9,Avalon Beach,2107,1400000,3.0,1,2.0,house
9,2019-04-09,10,Avalon Beach,2107,1210000,3.0,2,2.0,house


### First, let's examine the suburb and PropType Feature

In [5]:
data.groupby('suburb')['suburb'].agg("count").sort_values(ascending = False)

suburb
Castle Hill           1250
Strathfield           1248
Ermington             1248
Carlingford           1247
Auburn                1244
                      ... 
Nattai                   1
Marrickville Metro       1
Haymarket                1
Clarendon                1
Paradise Point           1
Name: suburb, Length: 685, dtype: int64

In [6]:
data.groupby('propType')['propType'].agg('count').sort_values(ascending = False)

propType
house                   170117
townhouse                15177
duplex/semi-detached      6722
villa                     5701
terrace                   1344
other                      392
warehouse                   40
acreage                     11
Name: propType, dtype: int64

### As we can see, it seems that most of the properties sold during the last 20 years in Sydney are by far Houses, followed by Townhouses.    
### Also, the most popular suburbs are Castle Hill, Strathfield and Ermington, closely followed by Carlingford and Auburn.     
### Now let's drop features which won't be useful or redundant

In [7]:
house_price = data.drop(['Id', 'postalCode'], axis = 'columns')
house_price.head(10)

,Date,suburb,sellPrice,bed,bath,car,propType
0,2019-06-19,Avalon Beach,1210000,4.0,2,2.0,house
1,2019-06-13,Avalon Beach,2250000,4.0,3,4.0,house
2,2019-06-07,Whale Beach,2920000,3.0,3,2.0,house
3,2019-05-28,Avalon Beach,1530000,3.0,1,2.0,house
4,2019-05-22,Whale Beach,8000000,5.0,4,4.0,house
5,2019-05-10,Bilgola Plateau,1610000,4.0,2,2.0,house
6,2019-04-13,Avalon Beach,1510000,3.0,1,2.0,house
7,2019-04-11,Avalon Beach,1500000,5.0,4,4.0,house
8,2019-04-11,Avalon Beach,1400000,3.0,1,2.0,house
9,2019-04-09,Avalon Beach,1210000,3.0,2,2.0,house


In [8]:
house_price.isnull().sum()

Date             0
suburb           0
sellPrice        0
bed            154
bath             0
car          18151
propType         0
dtype: int64

We can infer here that the nulls in the car columns mean that the property does not come with a garage, and that the null in the bed column means that this is either a warehouse or acreage; so we are going to keep these values as they won't affect the analysis

### Before moving on to the outlier removal part, let's split the date column in Year, Month, Day. Doing it this way will allow us later to do some efficient grouping.


In [9]:
house_price['Year'] = house_price['Date'].dt.year
house_price['Month'] = house_price['Date'].dt.month
house_price = house_price.drop(['Date'], axis = 'columns')


In [10]:
house_price.head()

,suburb,sellPrice,bed,bath,car,propType,Year,Month
0,Avalon Beach,1210000,4.0,2,2.0,house,2019,6
1,Avalon Beach,2250000,4.0,3,4.0,house,2019,6
2,Whale Beach,2920000,3.0,3,2.0,house,2019,6
3,Avalon Beach,1530000,3.0,1,2.0,house,2019,5
4,Whale Beach,8000000,5.0,4,4.0,house,2019,5


### Finally, let's reorder the columns for better readability

In [11]:
house_price = house_price[['Year', 'Month', 'suburb', 'propType', 'sellPrice', 'bed', 'bath', 'car' ]]

In [12]:
house_price.head(10)

,Year,Month,suburb,propType,sellPrice,bed,bath,car
0,2019,6,Avalon Beach,house,1210000,4.0,2,2.0
1,2019,6,Avalon Beach,house,2250000,4.0,3,4.0
2,2019,6,Whale Beach,house,2920000,3.0,3,2.0
3,2019,5,Avalon Beach,house,1530000,3.0,1,2.0
4,2019,5,Whale Beach,house,8000000,5.0,4,4.0
5,2019,5,Bilgola Plateau,house,1610000,4.0,2,2.0
6,2019,4,Avalon Beach,house,1510000,3.0,1,2.0
7,2019,4,Avalon Beach,house,1500000,5.0,4,4.0
8,2019,4,Avalon Beach,house,1400000,3.0,1,2.0
9,2019,4,Avalon Beach,house,1210000,3.0,2,2.0


### Outlier analysis and removal

#### Now that we've made sure that our dataset is cleaned up, it's time to remove outliers which may skew our results

#### Let's start with the bed column first.  

In [13]:
house_price['bed'].unique()

array([ 4.,  3.,  5.,  6.,  2.,  7.,  8.,  1., nan, 10.,  9., 12., 15.,
       19., 26., 20., 14., 24., 13., 30., 16., 36., 18., 23., 11., 17.,
       99., 22., 45., 21., 34.])

##### As we can see, there are some properties with 99, 45 or 36 bedrooms. This is either a human error or outliers which will skew our results, so let's analyse them and depending on the results, we will get rid of them.

In [14]:
house_price[house_price['bed'] > 30]

,Year,Month,suburb,propType,sellPrice,bed,bath,car
37025,2009,3,Neutral Bay,townhouse,6215000,36.0,24,24.0
106339,2013,2,Dangar Island,house,660000,99.0,1,4.0
170630,2013,3,Burwood,house,3640000,45.0,9,NaN
188811,2011,12,Birrong,house,445000,34.0,1,2.0


##### Since we don't have the total square footage information, we can't possibly know if this is due to human error or not.    
#### However, we can get rid of the 99, 45 and 34 beds row since we can infer that such a gigantic house would come with more bathrooms, a garage, or would ask a more expensive price.

In [15]:
house_price = house_price.drop(house_price[house_price.bed >= 30].index)

In [16]:
house_price[house_price.bed >30]

,Year,Month,suburb,propType,sellPrice,bed,bath,car


#### Now, let's do the same process for the number of bathrooms. If there are more than 30 bathrooms, we get rid of the row

In [17]:
house_price = house_price.drop(house_price[house_price.bath > 30].index)

In [18]:
house_price[house_price.bath > 30]

,Year,Month,suburb,propType,sellPrice,bed,bath,car


## Part 2: Feature Engineering and Dimensionality Reduction

### First, let's analyze how many suburbs we have

In [19]:
list_suburbs = house_price['suburb'].unique()
print(len(list_suburbs), list_suburbs[:10])

685 ['Avalon Beach' 'Whale Beach' 'Bilgola Plateau' 'Clareville'
 'Bilgola Beach' 'Blacktown' 'Marayong' 'Kings Park' 'Prospect'
 'Matraville']


#### As we can see, we have 685 different suburbs.    
#### If we were to create the dummy column by suburb, we would have 685 more columns, so we need to do some dimensionality reduction first.

In [20]:
number_by_suburb = house_price.groupby('suburb')['suburb'].agg('count').sort_values(ascending = False) #get the number of properties sold by suburb in descending order
print(number_by_suburb)

suburb
Castle Hill           1250
Strathfield           1248
Ermington             1248
Carlingford           1247
Auburn                1244
                      ... 
Nattai                   1
Marrickville Metro       1
Haymarket                1
Clarendon                1
Paradise Point           1
Name: suburb, Length: 685, dtype: int64


#### We have some suburbs which only appear once or twice, so we can group them together in order to reduce the number of dimension.    
#### Let's say that, if a suburb appears less than 10 times, we put it in a suburb named 'other'.

In [21]:
len(number_by_suburb[number_by_suburb <= 10] ) # we can get rid of 77 columns
len(number_by_suburb[number_by_suburb <= 20]) # we can get rid 108 columns

108

### Let's choose, as an arbitrary cutoff, less than 20 location. This means that on average, 1 property is sold in this suburb by year.

In [22]:
less_than_20_suburbs = number_by_suburb[number_by_suburb <= 20]
print(less_than_20_suburbs)

suburb
Lakesland             20
Lower Portland        20
Duffys Forest         20
South Turramurra      20
Elizabeth Bay         19
                      ..
Nattai                 1
Marrickville Metro     1
Haymarket              1
Clarendon              1
Paradise Point         1
Name: suburb, Length: 108, dtype: int64


In [23]:
house_price.suburb = house_price.suburb.apply(lambda x: 'other' if x in less_than_20_suburbs else x)

In [24]:
house_price.groupby('suburb')['suburb'].agg('count').sort_values(ascending = False)

suburb
Castle Hill      1250
Ermington        1248
Strathfield      1248
Carlingford      1247
Auburn           1244
                 ... 
St Leonards        22
Grose Wold         22
Luddenham          22
Port Hacking       21
Bilgola Beach      21
Name: suburb, Length: 578, dtype: int64

## Part 3: Visualization

### First, let's visualize the type of properties which sold the most.